In [ ]:
# NOTES:
# Why not use mixture of experts?

In [1]:
import os
import sys
import random
import pandas as pd
import numpy as np
from scipy.linalg import toeplitz
from copy import copy
import matplotlib.pyplot as plt
%matplotlib inline

# Geniuses that worked on hypertools did not update certain package and thus it produces warnings (they break jupyter lab)
import warnings
warnings.filterwarnings("ignore")

# Comment out if you don't want to see all of the values being printed (i.e. default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

current_dir = os.getcwd()
# utils_path = os.path.join(current_dir, '..', 'utils')
utils_path = os.path.join(current_dir, '../')
utils_abs_path = os.path.abspath(utils_path)
if utils_abs_path not in sys.path:
    sys.path.append(utils_abs_path)

import utils.get_data as get_data
# from impute_methods import *
from utils.impute_methods import impute_linear_interpolation

DATA_PATH = get_data.get_dataset_abspath()

training_setA_path = DATA_PATH + 'training_setA'
training_setB_path = DATA_PATH + 'training_setB'

In [2]:
def plot_heart_rate_data(df):
    plt.figure(figsize=(10, 6))
    dataset['HR'].hist(bins=50)
    plt.title('Distribution of Heart Rate')
    plt.xlabel('Heart Rate')
    plt.ylabel('Frequency')
    plt.show()
    
    # You can also get a quick statistical summary
    print(dataset['HR'].describe())
    

In [3]:
# Loads the dataset

# Sepsis related test values / variables / columns
sep_col = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST',
             'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
             'Bilirubin_total', 'Hct', 'Hgb', 'PTT', 'WBC', 'Platelets',
             'Bilirubin_direct', 'Fibrinogen']

# Continues Health Indicators
con_col = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2']

# The original way of getting data shouldn't work as there isn't a concept of individual patient file in it
# It just gets the data completely into a dataframe and each of the time data is one row
# dataset = get_data.get_dataset_as_df()

dataset, patient_id_map = get_data.get_dataset()

   20337
   40337
Dataset loaded into a MultiIndex DataFrame.


In [4]:
# for patient_id, file_name in patient_id_map.items():
#     print(type(dataset.loc[patient_id]))
#     break

In [5]:
columns_to_linearly_interpolate = [
    'HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Resp'
]
columns_to_ffill = [
    'Temp', 'Glucose', 'Potassium', 'Calcium', 
    'Magnesium', 'Chloride', 'Hct', 'Hgb', 'WBC', 'Platelets'
]
columns_to_drop = [
    'SepsisLabel', 'TroponinI'
]

In [6]:
def feature_missing_information(patient_data, columns):
    # temp_data holds the information from the patient file as well as the features that will be calculated
    temp_data = np.array(patient_data)

    # Calculate 3 features for each column, 2 respective of the frequency of NaN values and 1 respective of the change in recorded values
    for column in columns:
        data = np.array(patient_data[column])
        nan_pos = np.where(~np.isnan(data))[0]
        
        # Measurement frequency sequence
        interval_f1 = data.copy()
        # Measurement time interval
        interval_f2 = data.copy()

        # If all the values are NaN
        if (len(nan_pos) == 0):
            interval_f1[:] = 0
            temp_data = np.column_stack((temp_data, interval_f1))
            interval_f2[:] = -1
            temp_data = np.column_stack((temp_data, interval_f2))
        else :
            # Puts number of measurements into temp_data
            interval_f1[: nan_pos[0]] = 0
            for p in range(len(nan_pos)-1):
                interval_f1[nan_pos[p]: nan_pos[p+1]] = p + 1
            interval_f1[nan_pos[-1] :] = len(nan_pos)
            temp_data = np.column_stack((temp_data, interval_f1))

            # Puts the frequency of measurements into temp_data
            interval_f2[:nan_pos[0]] = -1
            for q in range(len(nan_pos) - 1):
                length = nan_pos[q+1] - nan_pos[q]
                for l in range(length):
                    interval_f2[nan_pos[q] + l] = l

            length = len(patient_data) - nan_pos[-1]
            for l in range(length):
                interval_f2[nan_pos[-1] + l] = l
            temp_data = np.column_stack((temp_data, interval_f2))

        # Differential features
        # These capture the change in values that have been recorded (quite simply as well but it should be just fine)
        diff_f = data.copy()
        diff_f = diff_f.astype(float)
        if len(nan_pos) <= 1:
            diff_f[:] = np.NaN
            temp_data = np.column_stack((temp_data, diff_f))
        else:
            diff_f[:nan_pos[1]] = np.NaN
            for p in range(1, len(nan_pos)-1):
                diff_f[nan_pos[p] : nan_pos[p+1]] = data[nan_pos[p]] - data[nan_pos[p-1]]
            diff_f[nan_pos[-1]:] = data[nan_pos[-1]] - data[nan_pos[-2]]
            temp_data = np.column_stack((temp_data, diff_f))
    
    return temp_data

In [7]:
def feature_slide_window(patient_data, columns):
    
    window_size = 6
    features = {}
    
    for column in columns:
        series = patient_data[column]

        features[f'{column}_max'] = series.rolling(window=window_size, min_periods=1).max()
        features[f'{column}_min'] = series.rolling(window=window_size, min_periods=1).min()
        features[f'{column}_mean'] = series.rolling(window=window_size, min_periods=1).mean()
        features[f'{column}_median'] = series.rolling(window=window_size, min_periods=1).median()
        features[f'{column}_std'] = series.rolling(window=window_size, min_periods=1).std()
        
        # For calculating std dev of differences, use diff() then apply rolling std
        diff_std = series.diff().rolling(window=window_size, min_periods=1).std()
        features[f'{column}_diff_std'] = diff_std

    # Convert the dictionary of features into a DataFrame
    features_df = pd.DataFrame(features)
    
    return features_df

In [8]:
def features_score(patient_data):
    """
    Gives score assocciated with the patient data according to the scoring systems of NEWS, SOFA and qSOFA
    """
    
    scores = np.zeros((len(patient_data), 8))
    
    for ii in range(len(patient_data)):
        HR = patient_data[ii, 0]
        if HR == np.nan:
            HR_score = np.nan
        elif (HR <= 40) | (HR >= 131):
            HR_score = 3
        elif 111 <= HR <= 130:
            HR_score = 2
        elif (41 <= HR <= 50) | (91 <= HR <= 110):
            HR_score = 1
        else:
            HR_score = 0
        scores[ii, 0] = HR_score

        Temp = patient_data[ii, 2]
        if Temp == np.nan:
            Temp_score = np.nan
        elif Temp <= 35:
            Temp_score = 3
        elif Temp >= 39.1:
            Temp_score = 2
        elif (35.1 <= Temp <= 36.0) | (38.1 <= Temp <= 39.0):
            Temp_score = 1
        else:
            Temp_score = 0
        scores[ii, 1] = Temp_score

        Resp = patient_data[ii, 6]
        if Resp == np.nan:
            Resp_score = np.nan
        elif (Resp < 8) | (Resp > 25):
            Resp_score = 3
        elif 21 <= Resp <= 24:
            Resp_score = 2
        elif 9 <= Resp <= 11:
            Resp_score = 1
        else:
            Resp_score = 0
        scores[ii, 2] = Resp_score

        Creatinine = patient_data[ii, 19]
        if Creatinine == np.nan:
            Creatinine_score = np.nan
        elif Creatinine < 1.2:
            Creatinine_score = 0
        elif Creatinine < 2:
            Creatinine_score = 1
        elif Creatinine < 3.5:
            Creatinine_score = 2
        else:
            Creatinine_score = 3
        scores[ii, 3] = Creatinine_score

        MAP = patient_data[ii, 4]
        if MAP == np.nan:
            MAP_score = np.nan
        elif MAP >= 70:
            MAP_score = 0
        else:
            MAP_score = 1
        scores[ii, 4] = MAP_score

        SBP = patient_data[ii, 3]
        Resp = patient_data[ii, 6]
        if SBP + Resp == np.nan:
            qsofa = np.nan
        elif (SBP <= 100) & (Resp >= 22):
            qsofa = 1
        else:
            qsofa = 0
        scores[ii, 5] = qsofa

        Platelets = patient_data[ii, 30]
        if Platelets == np.nan:
            Platelets_score = np.nan
        elif Platelets <= 50:
            Platelets_score = 3
        elif Platelets <= 100:
            Platelets_score = 2
        elif Platelets <= 150:
            Platelets_score = 1
        else:
            Platelets_score = 0
        scores[ii, 6] = Platelets_score

        Bilirubin = patient_data[ii, 25]
        if Bilirubin == np.nan:
            Bilirubin_score = np.nan
        elif Bilirubin < 1.2:
            Bilirubin_score = 0
        elif Bilirubin < 2:
            Bilirubin_score = 1
        elif Bilirubin < 6:
            Bilirubin_score = 2
        else:
            Bilirubin_score = 3
        scores[ii, 7] = Bilirubin_score
        
    return scores

In [9]:
def extract_features(patient_data):
    # Get the column with Sepsis Label as it is not the same for each row (check documentation)
    labels = np.array(patient_data['SepsisLabel'])
    patient_data = patient_data.drop(columns=columns_to_drop)

    # Gets information from the missing variables 
    # This can be useful as it shows the clinical judgment, the test has not been ordered 
    #                              (probably a good decision we should take into account)
    temp_data = feature_missing_information(patient_data, sep_col + con_col)
    temp = pd.DataFrame(temp_data)
    # To complete the data use forward-filling strategy
    temp = temp.fillna(method='ffill')
    # These are also the first set of features
    # In this configutation 99 (66 + 33 or 3 per column) features to be precise
    # They are also time indifferent
    features_A = np.array(temp)
    # The team did not use DBP, not sure why, might investigate this
    # columns = ['HR', 'O2Sat', 'SBP', 'MAP', 'Resp', 'DBP']
    
    # six-hour slide window statistics of selected columns
    columns = ['HR', 'O2Sat', 'SBP', 'MAP', 'Resp']
    features_B = feature_slide_window(patient_data, columns)

    # Score features based according to NEWS, SOFA and qSOFA
    features_C = features_score(features_A)
    
    features = np.column_stack([features_A, features_B, features_C])
    
    return features, labels

In [10]:
# Data Pre-processing
def preprecess_data(dataset):
    frames_features = []
    frames_labels = []
    
    for patient_id in set(dataset.index.get_level_values(0)):
        print(f"Processing data for patient ID: {patient_id}, File: {patient_id_map[patient_id]}", end='\r')
        
        patient_data = dataset.loc[patient_id]
    
        features, labels = extract_features(patient_data)
        features = pd.DataFrame(features)
        labels = pd.DataFrame(labels)
    
        frames_features.append(features)
        frames_labels.append(labels)

    data_features = np.array(pd.concat(frames_features))
    data_labels = (np.array(pd.concat(frames_labels)))[:, 0]
    
    # Randomly shuffle the data
    index = [i for i in range(len(data_labels))]
    np.random.shuffle(index)
    data_features = data_features[index]
    data_labels = data_labels[index]
    
    return data_features, data_labels

In [11]:
# Run the preprocess function
# features, labels = preprecess_data(dataset)
# print("Done with data pre-processing")

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [13]:
# Simple XGBoost model training

def train_model(X, y, kfold, save_dir):
    best_f1 = -1  # Initialize best score to a low value
    best_model = None  # Placeholder for the best model
    
    accuracies = []
    f1_scores = []
    
    for train_index, test_index in kfold.split(X):
        # Splitting the data for this fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Optionally, standardize features
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Initialize and train the XGBoost model
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)

        # Evaluate
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        accuracies.append(accuracy)
        f1_scores.append(f1)

        print()

        if f1 > best_f1:
            best_f1 = f1
            best_model = model
            
    # Display average scores
    print(f"Average Accuracy: {np.mean(accuracies):.4f}")
    print(f"Average F1 Score: {np.mean(f1_scores):.4f}")

    # After finding the best model, save it
    if best_model is not None:
        save_model_path = save_dir + 'simple_xgboost_model{}.mdl'.format(1)
        best_model.get_booster().save_model(fname=save_model_path)
        print(f"Best model saved with F1 score: {best_f1}")
    else:
        print("No model was saved.")
    

In [14]:
# Fully copied function to downsample data
# Reason is it is quite simple and provides a great explanation as to why it is needed
def downsample(data_set):
    """
    Using our feature extraction approach will result in over 1 million hours of data in the training process.
    However, only roughly 1.8% of these data corresponds to a positive outcome.
    Consequently, in order to deal with the serious class imbalance, a systematic way is provided by
    down sampling the excessive data instances of the majority class in each cross validation.
    """

    # Runs preprocessing here (combined with loading)
    x, y = preprecess_data(data_set)
    index_0 = np.where(y == 0)[0]
    index_1 = np.where(y == 1)[0]

    index = index_0[len(index_1): -1]
    x_del = np.delete(x, index, 0)
    y_del = np.delete(y, index, 0)
    index = [i for i in range(len(y_del))]
    np.random.shuffle(index)
    x_del = x_del[index]
    y_del = y_del[index]

    return x_del, y_del

In [15]:
# Normally only ~1.8% has sepsis label 1
# print(sum(labels)/len(labels))


kfold = KFold(n_splits=5, shuffle=True, random_state=np.random.seed(1337))

save_path = '../models/saved/'

X, y = downsample(dataset)

In [16]:
# Training the model

train_model(X, y, kfold, save_path)






Average Accuracy: 0.8638
Average F1 Score: 0.8660
Best model saved with F1 score: 0.8674225515578965


In [17]:
# Hyperparameter optimization for the actual model:

# Bayesian Optimization with the Tree-structured Parzen Estimator (TPE) algorithm
from hyperopt import hp, fmin, tpe, STATUS_OK

def BO_TPE(X_train, y_train, X_val, y_val):
    """
    Hyperparameter optimization using Bayesian Optimization with TPE.
    """
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)
    X_val_D = xgb.DMatrix(X_val)

    def objective(params):
        # Training and validation
        xgb_model = xgb.train(params, dtrain=train, num_boost_round=1000, evals=[(val, 'eval')],
                              verbose_eval=False, early_stopping_rounds=80)
        # Prediction
        y_vd_pred = xgb_model.predict(X_val_D)
        y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]
        # Evaluation
        loss = 1 - accuracy_score(y_val, y_val_class)

        return {'loss': loss, 'params': params, 'status': STATUS_OK}

    max_depths = [3, 4, 5, 6, 7, 8, 9, 10, 11]
    learning_rates = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.15, 0.2]
    subsamples = [0.5, 0.6, 0.7, 0.8, 0.9]
    colsample_bytrees = [0.5, 0.6, 0.7, 0.8, 0.9]
    reg_alphas = [0.0, 0.005, 0.01, 0.05, 0.1]
    reg_lambdas = [0.8, 1, 1.5, 2, 4]
    gammas = [0, 0.1, 0.2, 0.5, 1]
    min_child_weights = [1, 5, 10, 20]
    num_boost_rounds = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]
    
    # Hyperparameter space
    space = {
        'max_depth': hp.choice('max_depth', max_depths),
        'learning_rate': hp.choice('learning_rate', learning_rates),
        'subsample': hp.choice('subsample', subsamples),
        'colsample_bytree': hp.choice('colsample_bytree', colsample_bytrees),
        'reg_alpha': hp.choice('reg_alpha', reg_alphas),
        'reg_lambda': hp.choice('reg_lambda', reg_lambdas),
        'gamma': hp.choice('gamma', gammas),
        'min_child_weight': hp.choice('min_child_weight', min_child_weights),
        'num_boost_round': hp.choice('num_boost_round', num_boost_rounds)
    }

    # Optimization
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20)
    
    # Translate 'best' indices to actual values
    best_param = {
        'max_depth': max_depths[(best['max_depth'])],
        'learning_rate': learning_rates[(best['learning_rate'])],
        'subsample': subsamples[(best['subsample'])],
        'colsample_bytree': colsample_bytrees[(best['colsample_bytree'])],
        'reg_alpha': reg_alphas[(best['reg_alpha'])],
        'reg_lambda': reg_lambdas[(best['reg_lambda'])],
        'gamma': gammas[(best['gamma'])],
        'min_child_weight': min_child_weights[(best['min_child_weight'])],
        'num_boost_round': num_boost_rounds[(best['num_boost_round'])]
    }
    
    return best_param


In [18]:
# Actual model config used in the paper

def train_actual_model(k, X_train, y_train, X_val, y_val, save_dir):
    print("Hyperparameter optimization")
    # Finds the best hyperparameters
    best_param = BO_TPE(X_train, y_train, X_val, y_val)
    # Creates the XGBoost Classifier using the best found hyperparameters
    model = xgb.XGBClassifier(
        max_depth = best_param['max_depth'],
        eta = best_param['learning_rate'],
        n_estimators = best_param['num_boost_round'],
        subsample = best_param['subsample'],
        colsample_bytree = best_param['colsample_bytree'],
        reg_alpha = best_param['reg_alpha'],
        reg_lambda = best_param['reg_lambda'],
        gamma = best_param['gamma'],
        min_child_weight = best_param['min_child_weight'],
        objective = "binary:logistic"
    )
    # Fits the classifier
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], eval_metric='error', early_stopping_rounds=80, verbose=False)

    # Printing training AUC and accurace
    y_tr_pred = (model.predict_proba(X_train))[:, 1]
    train_auc = roc_auc_score(y_train, y_tr_pred)
    print('training dataset AUC: ' + str(train_auc))
    best = 0
    threshold = 0
    for t in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        y_tr_class = [0 if i <= t else 1 for i in y_tr_pred]
        acc = accuracy_score(y_train, y_tr_class)
        if (acc > best):
            best = acc
            threshold = t
    print('training dataset acc: ' + str(best) + ' using threshold of ' + str(threshold))

    # Priting validation AUC and accuracy
    y_vd_pred = (model.predict_proba(X_val))[:, 1]
    valid_auc = roc_auc_score(y_val, y_vd_pred)
    print('validation dataset AUC: ' + str(valid_auc))
    best = 0
    threshold = 0
    for t in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]
        acc = accuracy_score(y_val, y_val_class)
        if (acc > best):
            best = acc
            threshold = t
    print('validation dataset acc: ' + str(best) + ' using thresholld of ' + str(threshold))
    
    print(f"Using following parameters:\n\
            max_depth : {best_param['max_depth']}\n\
            eta (learning rate) : {best_param['learning_rate']}\n\
            n_estimators : {best_param['num_boost_round']}\n\
            subsample : {best_param['subsample']}\n\
            colsample_bytree : {best_param['colsample_bytree']}\n\
            reg_alpha : {best_param['reg_alpha']}\n\
            reg_lambda ; {best_param['reg_lambda']}\n\
            gamma : {best_param['gamma']}\n\
            min_child_weight : {best_param['min_child_weight']}\n\
            objective : 'binary:logistic'")
    
    save_model_path = save_dir + 'xgboost_model{}.mdl'.format(k + 1)
    model.get_booster().save_model(fname=save_model_path)
    

In [19]:
# Load the data

# We might do this each loop iteration
# Also consider changing the downsample function slightly 
X, y = downsample(dataset)

kfold = KFold(n_splits=5, shuffle=True, random_state=np.random.seed(1337))

In [20]:
# Run training loop

for fold, (train_indices, val_indices) in enumerate(kfold.split(X)):
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]

    train_actual_model(fold, X_train, y_train, X_val, y_val, save_path)

Hyperparameter optimization
100%|████████████████████████████████████████████| 20/20 [08:15<00:00, 24.76s/trial, best loss: 0.07092325602220828]
training dataset AUC: 0.9969720618313376
training dataset acc: 0.9733801997044732 using threshold of 0.5
validation dataset AUC: 0.9549961329098515
validation dataset acc: 0.8920032237843647 using thresholld of 0.1
Using following parameters:
            max_depth : 8
            eta (learning rate) : 0.2
            n_estimators : 100
            subsample : 0.9
            colsample_bytree : 0.5
            reg_alpha : 0.01
            reg_lambda ; 1
            gamma : 0
            min_child_weight : 1
            objective : 'binary:logistic'
Hyperparameter optimization
100%|████████████████████████████████████████████| 20/20 [12:09<00:00, 36.49s/trial, best loss: 0.06116235336258624]
training dataset AUC: 0.999988028368968
training dataset acc: 0.9980522097344736 using threshold of 0.5
validation dataset AUC: 0.9815645608453695
validatio

In [21]:
"""
Notes:
- Can't really use mean values to substitute for NaN values since the values have increased probability of being unhealthy since the test was ordered by a doctor

"""

"\nNotes:\n- Can't really use mean values to substitute for NaN values since the values have increased probability of being unhealthy since the test was ordered by a doctor\n\n"

In [22]:
import os
import sys
import random
import pandas as pd
import numpy as np
from scipy.linalg import toeplitz
from copy import copy
import matplotlib.pyplot as plt
%matplotlib inline

# Geniuses that worked on hypertools did not update certain package and thus it produces warnings (they break jupyter lab)
import warnings
warnings.filterwarnings("ignore")

# Comment out if you don't want to see all of the values being printed (i.e. default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

current_dir = os.getcwd()
# utils_path = os.path.join(current_dir, '..', 'utils')
utils_path = os.path.join(current_dir, '../')
utils_abs_path = os.path.abspath(utils_path)
if utils_abs_path not in sys.path:
    sys.path.append(utils_abs_path)

import utils.get_data as get_data
# from impute_methods import *
from utils.impute_methods import impute_linear_interpolation

In [23]:
import torch
import torch.nn as nn

class TimeSeriesTransformer1(nn.Module):
    def __init__(self, num_features, num_blocks=1, d_model=64, nhead=4, dim_feedforward=256, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                                   dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_blocks)
        self.output_layer = nn.Linear(d_model, 1)  # Outputting a probability for each time step

    def forward(self, src):
        src = self.embedding(src)  # (Batch, Seq, Features) -> (Batch, Seq, d_model)
        output = self.transformer_encoder(src)
        output = self.output_layer(output)
        return torch.sigmoid(output)  # Apply sigmoid to get probabilities


In [24]:
dataset, patient_id_map = get_data.get_dataset()

   20337
   40337
Dataset loaded into a MultiIndex DataFrame.


In [25]:
# First lets experiment with only raw data 
# We have to however impute NaN values since Neural Networks can't (natively) handle them

columns_to_linearly_interpolate = [
    'HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Resp'
]

# Feel free to omit this (EXPERIMENTAL)
# Normilize the dataset
if True:
    # Check if multiindex_df is indeed a MultiIndex DataFrame
    if isinstance(dataset.index, pd.MultiIndex):
        # Normalize each patient's data
        # This will apply z-score normalization per patient per feature
        normalized_data = dataset.groupby(level=0).transform(lambda x: (x - x.mean()) / x.std())
    
        # Optionally fill NaN values if they are created by division by zero in cases where std is zero
        normalized_data = normalized_data.fillna(0)
    
        # If you need to replace the old DataFrame with the new, normalized one
        dataset.update(normalized_data)
    else:
        print("The dataframe does not have a MultiIndex as expected.")

    
# Linear Interpolation
print("Linearly interpolating:")
for col in columns_to_linearly_interpolate:
    dataset = impute_linear_interpolation(dataset, col)
    print(col)
print("Done")



Linearly interpolating:
HR
O2Sat
SBP
MAP
DBP
Resp
Done


In [ ]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# Assuming 'dataset' has been appropriately preprocessed and split
features = dataset.drop('SepsisLabel', axis=1).values
labels = dataset['SepsisLabel'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32).clamp(0, 1) 
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).clamp(0, 1)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

# Initialize the model
num_features = X_train.shape[1]
model = TimeSeriesTransformer(num_features)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

    # Validation step
    model.eval()
    with torch.no_grad():
        valid_loss = []
        for data, target in val_loader:
            output = model(data)
            loss = criterion(output.squeeze(), target)
            valid_loss.append(loss.item())
        print(f'Validation Loss: {np.mean(valid_loss)}')

# Save the model
torch.save(model.state_dict(), 'time_series_transformer_model.pth')
print("Model saved successfully.")

TimeSeriesTransformer(
  (embedding): Linear(in_features=40, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

Epoch 1, Loss: 0.05951389279103011


TimeSeriesTransformer(
  (embedding): Linear(in_features=40, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

Validation Loss: 0.057526902803787174
Epoch 2, Loss: 0.05738054602967546


TimeSeriesTransformer(
  (embedding): Linear(in_features=40, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

...

Validation Loss: 0.05624296069827712
Epoch 6, Loss: 0.056289507521573974


TimeSeriesTransformer(
  (embedding): Linear(in_features=40, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

Validation Loss: 0.05640595956457657
Epoch 7, Loss: 0.056162805683411754


TimeSeriesTransformer(
  (embedding): Linear(in_features=40, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

Validation Loss: 0.056663643000764534


In [ ]:
import torch

In [ ]:
import torch.nn.functional as F

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, num_blocks=6, d_model=128, nhead=8, dim_feedforward=512, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        # Embedding layer expanded to take original features + missingness indicators
        self.embedding = nn.Linear(num_features * 2, d_model)  
        self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, d_model))  # Assuming max sequence length of 1000
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                                   dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_blocks)
        self.output_layer = nn.Linear(d_model, 1)
        
    def forward(self, src, src_mask=None):
        src = self.embedding(src)
        seq_length = src.size(1)
        src += self.positional_encoding[:, :seq_length]
        output = self.transformer_encoder(src, src_key_padding_mask=src_mask)
        output = self.output_layer(output)
        return torch.sigmoid(output)

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

# Prepare data and add missingness indicators
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train, X_val = add_nan_indicators(X_train), add_nan_indicators(X_val)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

# Create DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

# Initialize the model
model = TimeSeriesTransformer(num_features=X_train.shape[1] // 2)  # Assuming features were doubled to account for indicators
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# To keep all the data focus on the minority class (sepsis = 1)
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.BCELoss(weight=class_weights[1])  # Focus more on the minority class
criterion = nn.BCELoss()

# Train the model
num_epochs = 30
best_auroc = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            preds = output.squeeze()
            all_preds.extend(preds.tolist())
            all_targets.extend(target.tolist())

    # Calculate AUROC
    auroc = roc_auc_score(all_targets, all_preds)
    if auroc > best_auroc:
        best_auroc = auroc
        # Save model and predictions for the best model based on AUROC
        torch.save(model.state_dict(), 'best_model.pth')

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Validation AUROC: {auroc}')